In [1]:
import torchtext
from torchtext.vocab import Vectors, GloVe
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from hw1.models import *

In [2]:
# Our input $x$
TEXT = torchtext.data.Field()
    
# Our labels $y$
LABEL = torchtext.data.Field(sequential=False)


train, val, test = torchtext.datasets.SST.splits(
    TEXT, LABEL,
    filter_pred=lambda ex: ex.label != 'neutral')

# Build vocab
TEXT.build_vocab(train)
LABEL.build_vocab(train)

train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
(train, val, test), batch_size=10, device=-1)

# Build the vocabulary with word embeddings
url = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.simple.vec'
TEXT.vocab.load_vectors(vectors=Vectors('wiki.simple.vec', url=url))

In [14]:
model = CBOW(TEXT, LABEL)
for p in model.parameters():
    if p.requires_grad:
        print(p)
print(model.embeddings.weight[:10,:10])
trainer = TextTrainer(TEXT, LABEL, model)
trainer.train(train_iter, num_iter=1000, skip_iter=100)
print(model.embeddings.weight[:10,:10])

Parameter containing:
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0671 -0.0058 -0.0793  ...   0.0308  0.0540  0.0882
-0.2525 -0.0505 -0.1246  ...   0.0678  0.0826  0.3026
          ...             ⋱             ...          
 0.1253 -0.1663  0.2053  ...   0.0221 -0.5225 -0.0705
-0.0021  0.0008 -0.0020  ...  -0.0006  0.0027 -0.0001
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 16284x300]

Parameter containing:

Columns 0 to 9 
1.00000e-02 *
  3.3217  4.4757 -3.9592 -4.9604  3.6914  2.3955  0.5458  1.0182  0.4738  4.0487
  0.0547 -2.2727 -3.4778 -1.7274  0.5824  1.8603 -4.9241  0.4656  4.5720  0.5508
 -2.7608  4.3582  4.9227 -1.1431  1.2861  1.0787 -4.8951  5.1399 -5.3310  4.2848

Columns 10 to 19 
1.00000e-02 *
  4.0068 -5.3642  3.7567 -0.4841 -4.6548  4.6807  3.2525 -0.3577 -2.7947 -4.8811
  4.2396  1.5764 -2.6887 -2.0928 -1.0368  2.1105 -4.0316  3.7971  1.4540  2.9874
 -0.7472 -1.6380  2.9901  3.2048 -1.0400 -4.9838 -3.7554  1.2413  5.41

Iteration 100, loss: 1.694317
Iteration 200, loss: 0.437575
Iteration 300, loss: 1.140245
Iteration 400, loss: 0.465758
Iteration 500, loss: 0.449803
Iteration 600, loss: 1.286211
Iteration 700, loss: 0.642169
Iteration 800, loss: 0.848213
Iteration 900, loss: 0.715434
Variable containing:
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
-0.0968 -0.0231 -0.0775  0.1214 -0.0481 -0.0954  0.0662  0.0168 -0.0572 -0.0308
-0.4070 -0.0705 -0.2034  0.1931 -0.2440 -0.1833  0.0393  0.1389 -0.2937 -0.2520
-0.1767  0.0833 -0.3064  0.3803 -0.0618 -0.1790  0.1391  0.0540 -0.2477 -0.1820
-0.3424 -0.0573 -0.4077  0.4390 -0.0444 -0.2406  0.0932  0.0659 -0.1966 -0.1798
-0.1071 -0.0269 -0.1442  0.2953 -0.1712 -0.1028  0.0245 -0.1460 -0.1107 -0.0994
-0.1101 -0.2306 -0.1144  0.1689 -0.0556 -0.2764  0.0735  0.1391 -0.0789 -0.0109
-0.2102 -0.0145 -0.1591  0.0353 -0.0475 -0.1037 -0.0157  0.1019 -0.3215 -0.1522
-0.1239  0.1271 -0.1349  0.2067  0.0580  0.1208  0.0411 -0.0122 -0.05

In [12]:
class CBOW(nn.Module):
    def __init__(self, TEXT, LABEL):
        super(CBOW, self).__init__()
        # Embeddings vectors (should be trainable); [V, d]
        # TODO: is default for requires_grad True?
        self.embeddings = nn.EmbeddingBag(TEXT.vocab.vectors.size()[0],
                                          TEXT.vocab.vectors.size()[1],
                                          mode='sum')
        self.embeddings.weight = nn.Parameter(TEXT.vocab.vectors, requires_grad=True)
        
        # Linear layer
        self.linear = nn.Linear(TEXT.vocab.vectors.size()[1], len(LABEL.vocab))
        
    # Here bow is [len-of-sentence, N] -- it is an integer matrix
    def forward(self, bow):
        bow_features = self.embeddings(bow)
        return F.log_softmax(self.linear(bow_features), dim=1)

In [5]:
class LogisticRegression2(nn.Module):
    def __init__(self, TEXT, LABEL):
        super(LogisticRegression2, self).__init__()
        # Embeddings vectors (should be trainable); [V, d]
        # TODO: is default for requires_grad True?
        self.embeddings = nn.EmbeddingBag(len(TEXT.vocab),
                                          len(TEXT.vocab),
                                          mode='sum')
        self.embeddings.weight = nn.Parameter(torch.eye(len(TEXT.vocab)),
                                              requires_grad=False)
        # Linear layer
        self.linear = nn.Linear(len(TEXT.vocab), len(LABEL.vocab))
        
    # Here bow is [len-of-sentence, N] -- it is an integer matrix
    def forward(self, bow):
        bow_features = self.embeddings(bow)
        return F.log_softmax(self.linear(bow_features), dim=1)